## Intro to SciJava Ops

Note that for this tutorial we assume this notebook was started from the provided `environment.yml` which ensures [scyjava](https://github.com/scijava/scyjava) and its dependencies are included in this python environment.

Our first step is to use `scyjava` to point to the SciJava Ops tutorial endpoint, which gives us access to all the ops included in SciJava Ops and SciJava Ops Image:

In [1]:
from scyjava import config, jimport
config.endpoints.append('org.scijava:scijava-ops-tutorial:0-SNAPSHOT')

Next we use the `gateways` module to set up `OpEnvironment` and `OpGateway` variables. These will be our entry points into matching and calling Ops.

In [2]:
import gateways as g

env = g.env
ops = g.ops

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


In the rest of this tutorial we will open a remote image and use ops to apply a mean filter, multiply it by the original to improve the signal-to-noise ratio, and write the result to disk.

First, we will import a few necessary classes - SCIFIO's IO module for reading/writing, and ImgLib2 classes required for the ops.

In [3]:
io = jimport('io.scif.img.IO')
Rect = jimport('net.imglib2.algorithm.neighborhood.RectangleShape')
IntType = jimport('net.imglib2.type.numeric.integer.IntType')
OOBFactory = jimport('net.imglib2.outofbounds.OutOfBoundsBorderFactory')

Next we'll use specify an output path using `ipywidgets`. By default it will save to the same directory as this notebook.

In [4]:
import ipywidgets, IPython.display

path = ipywidgets.Text(
    description='Output file:',
    value='lymp.tif')

IPython.display.display(path)

Text(value='lymp.tif', description='Output file:')

Now we open our sample image

In [5]:
img = io.open("https://imagej.net/images/lymp.tif").getImg()

We are going to run our Ops as Computers. This involves providing pre-allocated output containers that are populated by the ops. Here we create an empty image of the same type/dimensions to hold our mean value.

In [10]:
Discoverer = jimport('org.scijava.discovery.Discoverer')
Discoverer.
Discoverer.all()

TypeError: No matching overloads found for *static* org.scijava.discovery.Discoverer.all(), options are:
	public static java.util.List org.scijava.discovery.Discoverer.all(java.util.function.Function)



Now we can apply the mean filter to our input `img` and save it in `mean`. Note the `out=mean` syntax: this is what indicates the call is a Computer Op.

In [7]:
ops.filter.mean(img, Rect(5, False), OOBFactory(), out=mean)

AttributeError: 'OpGateway' object has no attribute 'filter'

Now we're going to multiply the input `img` and our `mean`-filtered image. This will give us a result that has an improved signal-to-noise ratio - but it could also result in pixel values that go beyond the default 8-bit limits. To ensure our final result will not overflow in this way, we create an empty `Int`-type image to hold the multiplied pixels.

In [ ]:
result = ops.create.img(img, IntType())

Now we can call our multiplication Op, again as a Computer.

In [ ]:
ops.math.multiply(img, mean, out=result)

And finally we save the result!

In [ ]:
import os
if not os.path.isfile(path.value):
    io.save(path.value, result)

Note that because of the `Int`-typing, this image may not render properly by standard previews. Open it in an application like [Fiji](https://imagej.net/Fiji) to get proper rendering.